In [1]:
from pathlib import Path
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches

from sklearn.linear_model import RidgeCV,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import flammkuchen as fl

In [2]:
from fig_5_helper import catname, bout_cols
bout_cat =np.arange(0,11,1)

In [ ]:
SAVE_FIGS=True
fig_dir = Path(r'\\portulab.synology.me\data\Kata\Figures_Paper\Figure_5\fig_5')
fig_dir.mkdir(exist_ok=True, parents=True)

## Load data

In [ ]:
master_path = Path(r'\\portulab.synology.me\data\Kata\Processed_Data\fin_tail_nn')
ds = np.load(Path(master_path /'fin_data'/'model_data_test.npz'))
print(*ds.files,sep="\n")

In [ ]:
ds = np.load(Path(master_path /'fin_data'/'model_data_test.npz'))
ds['reconstructions'].shape, ds['tails'].shape, ds['model_targets'].shape, ds['inputs'].shape,  ds['fins'].shape, ds['latents'].shape

In [ ]:
# pull out bout features
swim_cat = ds['clusters'] # bout type
swim_direction = ds['tail_dir'] # direction
proba = ds['proba'] # direction

# pull out bout tensors
fins = ds['fins'][:-1,:,:]
tails = ds['tails'][:-1,:,:]
latents = ds['latents']
inputs = ds['inputs']
recons = ds['reconstructions']

# dims
n_trials, n_segs, n_tps = recons.shape
_, n_fins, _ = fins.shape
_, n_latents, _ = latents.shape
_, n_inputs, _ = inputs.shape

print(f"n trials: {n_trials}, n tps: {n_tps}, n fins: {n_fins}, n latents: {n_latents}")

tt_ = np.arange(n_tps)*10/7

## Fit tail latents to fin

In [ ]:

# Assuming latents as features (X) and fins as target (y)
print(latents.shape, fins.shape)
X = latents.reshape(latents.shape[0], -1)  # Flatten the latents array
y = fins.reshape(fins.shape[0], -1)  # Flatten the fins array
print(X.shape, y.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, y_train = X, y

# Define the Ridge regression model with cross-validation
alphas = [1e-4, 1e-3, 0.01, 0.1, 1.0,]  # List of alphas to cross-validate
ridge_cv = RidgeCV(alphas=alphas, store_cv_results=True)

# Fit the model
ridge_cv.fit(X_train, y_train)

# Predict on the test set
y_pred = ridge_cv.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Best alpha: {ridge_cv.alpha_}')

In [ ]:
# np.random.seed(2)
np.random.seed(3)

ytrain_pred = ridge_cv.predict(X_train)
sk_ytrain_=ytrain_pred.reshape(ytrain_pred.shape[0],n_fins,n_tps).transpose(0,2,1)
sk_ytrain_true_=y_train.reshape(ytrain_pred.shape[0],n_fins,n_tps).transpose(0,2,1)


In [ ]:
print(f"Train mse: {mean_squared_error(y_train, ytrain_pred):.04f}\tTest mse: {mean_squared_error(y_test, y_pred):.04f}")

## Calculate r-score for all bout

In [ ]:
# Predict on the test set
ridge_cv.fit(X, y)
y_pred_ = ridge_cv.predict(X)

sk_y_=y_pred_.reshape(y_pred_.shape[0],n_fins,n_tps).transpose(0,2,1)
sk_ytrue_=y.reshape(y_pred_.shape[0],n_fins,n_tps).transpose(0,2,1)

r_vals = np.array([r2_score(tr, pr) for tr, pr in zip(y, y_pred_)])
r_vals.shape

In [ ]:
means, errs, seq1, seq2 = [], [], [], []
for a_cat in range(len(bout_cat)):
    if a_cat ==10:
        print (a_cat)
        ids_ = np.where(swim_cat==a_cat)[0][:-1]
        print(catname[a_cat],ids_.shape)
        # print(ids_)
        means.append(np.mean(r_vals[ids_]))
        seq1.append(r_vals[ids_][r_vals[ids_]>0.0])
        seq2.append(np.clip(r_vals[ids_],0,None))
        errs.append(np.std(r_vals[ids_]))
        
    else:
        print (a_cat)
        ids_ = np.where(swim_cat==a_cat)[0]
        print(catname[a_cat],ids_.shape)
        # print(ids_)
        means.append(np.mean(r_vals[ids_]))
        seq1.append(r_vals[ids_][r_vals[ids_]>0.0])
        seq2.append(np.clip(r_vals[ids_],0,None))
        errs.append(np.std(r_vals[ids_]))
    
means, errs = np.array(means), np.array(errs)

In [ ]:
fl.save(master_path/ 'R2_sample.h5', seq2)

### Result plot

In [ ]:
proba_means, proba_errs= [], []
for a_cat in range(len(bout_cat)):
    if a_cat ==10:
        print (a_cat)
        ids_ = np.where(swim_cat==a_cat)[0][:-1]
        print(catname[a_cat],ids_.shape)
        proba_means.append(np.mean(proba[ids_]))
        proba_errs.append(np.std(proba[ids_]))
        
    else:
        print (a_cat)
        ids_ = np.where(swim_cat==a_cat)[0]
        print(catname[a_cat],ids_.shape)
        proba_means.append(np.mean(proba[ids_]))
        proba_errs.append(np.std(proba[ids_]))
    
proba_means, proba_errs = np.array(proba_means), np.array(proba_errs)

### Plot the fins with r2-score

In [ ]:
lw=0.5
cm = 1/2.54  # centimeters in inches
f_size = 6
dpi= 600
plt.rcParams.update({'font.size': f_size})

In [ ]:
np.random.seed(1)
fig, ax = plt.subplots(facecolor='None',figsize=(10.5*cm, 3*cm))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])

G = gridspec.GridSpec(3,len(np.unique(swim_cat)), height_ratios=[1,1,1])
ax0 = {}
for i,b in enumerate(bout_cat):
    ax0 = plt.subplot(G[0,i])
    ax0.set_title(catname[i],fontsize=f_size, c='k' ) #bout_cols[i]) 
    id = np.where(swim_cat==b)[0]
    if len(id)>0:
        id_ = np.random.choice(id)
        ax0.plot(sk_ytrue_[id_],color='k', lw=lw)
        ax0.plot(sk_y_[id_],color='r', lw=lw)
    ax0.set(xticks=[], yticks=[], xlim=(0,160), ylim=(-1.5,1.5),)
    for sp in ['top','bottom','left','right']:
        ax0.spines[sp].set_color('none')  # Turns off the spines
        
    ax1 = plt.subplot(G[1,i])
    if len(id)>0:
        [ax1.plot(tails[id_,i_,:], color="k", lw=lw) for i_ in range(7)]
  
    ax1.set(xticks=[], yticks=[], xlim=(0,160), ylim=(-4,4),)
    for sp in ['top','bottom','left','right']:
        ax1.spines[sp].set_color('none')  # Turns off the spines
    if seq1[i].size==0:
        continue
    
    ax2 = plt.subplot(G[2,i])
    parts = ax2.violinplot(seq1[i], showmeans=False, showmedians=True, widths=0.5)
    parts['bodies'][0].set_facecolor(bout_cols[b])
    parts['bodies'][0].set_edgecolor(bout_cols[b])
    parts['bodies'][0].set_alpha(0.7)
    parts['cmedians'].set_color('black')
    parts['cmedians'].set_linewidth(0.5)
    parts['cmins'].set_color('black')
    parts['cmaxes'].set_color('black')
    parts['cmins'].set_linewidth(0.5)
    parts['cmaxes'].set_linewidth(0.5)
    parts['cbars'].set_color('black')
    parts['cbars'].set_linewidth(0.5)
    
    ax2.set(yticks=[0, 0.5, 1])
    ax2.set_xticks(np.arange(0, 3))
    ax2.set(xticks=[], xlabel=catname[b], ylim=(0,1))
    for sp in ['top','right']:
        ax2.spines[sp].set_color('none')  # Turns off the spines
    
    # Remove y-tick labels from every subplot except the first one
    if i != 0:
        ax2.set_yticklabels([])

# Create custom legend patches
ground_truth_patch = mpatches.Patch(color='k', label='Ground Truth')
predicted_patch = mpatches.Patch(color='r', label='Predicted')

# Add the patches to the legend
ax0.legend(handles=[ground_truth_patch, predicted_patch], loc='upper center', bbox_to_anchor=(-1.5, 2.4),ncol=2,
           handleheight=0.3,  # Adjust handle height
           handlelength=1)    # Adjust handle length


ax0 = plt.subplot(G[0,0])
ax0.set_ylabel("Fin", fontsize=f_size)
ax1 = plt.subplot(G[1,0])
ax1.set_ylabel("Tail", fontsize=f_size)
ax2 = plt.subplot(G[2,0])
ax2.set_ylabel("$R^{2}$ Score", fontsize=f_size)

plt.tight_layout()

fig.savefig(fig_dir / "r2_fins_w_bouts_mod.pdf", bbox_inches='tight',dpi=dpi, transparent=True)
fig.savefig(fig_dir / "r2_fins_w_bouts_mod.png", bbox_inches='tight',dpi=dpi, transparent=True)